In [1]:
!nvidia-smi
!which python | grep DYY

Mon Sep 16 16:20:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   42C    P0              44W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
torch.manual_seed(3407)
torch.backends.cudnn.deterministic = True

# Model

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class SinousEmbedding(nn.Module):
#     def __init__(self, dim) -> None:
#         super().__init__()
#         assert dim%2==0,NotImplementedError()
#         self.angles = (1000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
#         self.angles.requires_grad_(False)
#     def forward(self,x):
#         angles = torch.einsum('m,i->im',self.angles,x.float())
#         return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

# class DDPM(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
#         self.in_size = 28 * 28
#         self.t_embedding_dim = 256
#         self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
#         self.up = nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(784+self.t_embedding_dim,64),
#                 nn.ReLU(),
#             ),
#             nn.Sequential(
#                 nn.Linear(64,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#         ])
#         self.middle = nn.ModuleList([
#             nn.Linear(32,32),
#             # nn.LeakyReLU(0.1),
#         ])
#         self.down= nn.ModuleList([
#             nn.Sequential(
#                 nn.Linear(32,32),
#                 nn.ReLU(),
#             ),
#             # nn.Sequential(
#             #     nn.Linear(256,256),
#             #     # nn.LeakyReLU(0.1),
#             # ),
#             nn.Sequential(
#                 nn.Linear(32,64),
#                 nn.ReLU(),
#             ),
#         ])
#         self.end_mlp = nn.Linear(64,784)
#         self.apply_init()

#     def apply_init(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.xavier_normal_(m.weight)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self,x,t):
#         x = x.reshape(-1,784)
#         ttensor = self.t_embedding(t) # [batch, 256]
#         batch = x.shape[0]
#         xc = x.clone()
#         ups = []
#         x = torch.cat((x,ttensor),dim=-1)
#         for ly in self.up:
#             x = ly(x)
#             ups.append(x.clone())
#         for ly in self.middle:
#             x = ly(x)
#         for ly in self.down:
#             x = ly(x) + ups.pop()

#         x = self.end_mlp(x)
#         x = (x + xc)
#         return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SinousEmbedding(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        assert dim%2==0,NotImplementedError()
        self.angles = (1000.**(-2/dim))**torch.arange(1,dim//2+1,1,dtype=torch.float).cuda()
        self.angles.requires_grad_(False)
    def forward(self,x):
        angles = torch.einsum('m,i->im',self.angles,x.float())
        return torch.cat((torch.sin(angles),torch.cos(angles)),dim=1)

class F_x_t(nn.Module):

    def __init__(self,in_channels,out_channels,out_size,kernel_size=3,t_shape=64) -> None:
        super().__init__()
        # self.t_channels = out_channels // 2
        # self.conv_channels = out_channels - self.t_channels
        self.t_channels = out_channels
        self.conv_channels = out_channels
        self.conv = nn.Conv2d(in_channels, self.conv_channels, kernel_size=kernel_size, padding=kernel_size//2)
        self.out_size = out_size
        self.fc = nn.Linear(t_shape, self.t_channels)
        # self.fc = nn.Embedding(t_shape, self.t_num)

    def forward(self, x, t):
        if self.t_channels == 0:
            raise NotImplementedError()
            return self.conv(x)
        # return torch.cat([self.conv(x),self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)],dim=1).relu()
        return (self.conv(x) + self.fc(t).unsqueeze(-1).unsqueeze(-1).expand(t.shape[0], self.t_channels, self.out_size, self.out_size)).relu()

class DDPM(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.t_embedding_dim = 32
        self.t_embedding = SinousEmbedding(dim=self.t_embedding_dim)
        self.up= nn.ModuleList([
            F_x_t(in_channels=1,out_channels=32,out_size=32,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=32,out_channels=64,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=64,out_channels=128,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim),
            # F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=1,t_shape=self.t_embedding_dim),
        ])
        self.middle = nn.ModuleList([
            # nn.Identity()
            F_x_t(in_channels=128,out_channels=128,out_size=4,kernel_size=1,t_shape=self.t_embedding_dim),
        ])
        self.down= nn.ModuleList([
            # F_x_t(in_channels=128,out_channels=128,out_size=2,kernel_size=1,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=128,out_channels=64,out_size=8,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=64,out_channels=32,out_size=16,kernel_size=3,t_shape=self.t_embedding_dim),
            F_x_t(in_channels=32,out_channels=16,out_size=32,kernel_size=3,t_shape=self.t_embedding_dim),
        ])
        # self.end_mlp = nn.Conv2d(32,1,kernel_size=3,padding=1)
        self.end_mlp = nn.Conv2d(16,1,kernel_size=1)
        self.apply_init()
    
    def apply_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self,x,t):
        x = x.reshape(-1,1,28,28)
        x = F.pad(x,(2,2,2,2),mode='constant',value=0)
        ttensor = self.t_embedding(t) # [batch, 256]
        batch = x.shape[0]
        # xc = x.clone()
        ups = []
        for ly in self.up:
            x = ly(x,ttensor)
            ups.append(x.clone()) # append: 28x28, 14x14
            x = nn.AvgPool2d(2)(x)
        for ly in self.middle:
            x = ly(x,ttensor)
        for ly in self.down:
            x = nn.Upsample(scale_factor=2)(x) + ups.pop() # 14x14, 28x28
            x = ly(x,ttensor)
            # x = nn.Upsample(scale_factor=2)(x) + ups.pop()
        x = self.end_mlp(x)
        x = x[:,:,2:30,2:30]
        return x.reshape(batch,28*28)

# Train

In [7]:
import sys
import os

parent_dir = os.path.abspath('/home/zhh24/DeepLearning')

sys.path.append(parent_dir)
print('appended',parent_dir)

import utils

from tqdm import tqdm
import torch
import torch.nn.functional as F
import torchvision.utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

mnist = utils.MNIST(batch_size=512)
train_loader = mnist.train_dataloader
valid_loader = mnist.valid_dataloader
T=500
beta1=3e-4 # variance of lowest temperature
betaT=2e-2 # variance of highest temperature
# step = torch.log(torch.tensor(betaT/beta1))/(T-1)
# betas = beta1 * torch.exp(step*torch.arange(T,dtype=torch.float).to(device))
step = (betaT-beta1)/(T-1)
betas = torch.arange(T,dtype=torch.float,device=device) * step + beta1


alphas = 1-betas
alpha_bars = alphas.clone()
for i in range(1,T):
    alpha_bars[i] *= alpha_bars[i-1]
print(alpha_bars)
# print(alphas)

sqrt = torch.sqrt
sigmas = sqrt(betas * (1-alpha_bars / alphas)/(1-alpha_bars))

@torch.no_grad()
def sample(model:DDPM,save_dir):
    x = torch.randn([100,784]).to(device)
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*sigmas[t]
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
    grid = torchvision.utils.make_grid(post_process(x).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)

@torch.no_grad()
def visualize(model,save_dir):
    x = torch.randn([10,784]).to(device)
    x_history = []
    for t in range(T-1,-1,-1):
        sigmaz = torch.randn_like(x)*((betas[t])**0.5).to(device)
        if t==0:
            sigmaz = 0
        x = (x-(1-alphas[t])/(sqrt(1-alpha_bars[t]))*model(x,t*torch.ones(x.shape[0],dtype=torch.long,device=device)))/(sqrt(alphas[t]))+sigmaz
        # x = torch.clamp(x,0,1)
        x_history.append(x)
    # print('cat.shape',torch.cat(x_history,dim=0).shape)
    grid = torchvision.utils.make_grid(post_process(torch.cat(x_history,dim=0)[3::4,...]).reshape(-1,1,28,28).cpu(), nrow=10)
    torchvision.utils.save_image(grid, save_dir)
    print('Saved visualize to',os.path.abspath(save_dir))

def pre_process(x):
    # do the logit transform
    # return (torch.log(x+1e-3)-torch.log(1-x+1e-3))
    return (x+1)/2

def post_process(x):
    # return torch.sigmoid(x)
    return x*2-1

def train(epochs,model:DDPM,optimizer,eval_interval=5):
    for epoch in range(epochs):
        model.train()
        with tqdm(train_loader) as bar:
            losses = []
            for x,_ in bar:
                x = pre_process(x.to(device))
                epss = torch.randn_like(x).reshape(-1,784).to(device)
                ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                alpha_tbars = alpha_bars[ts]
                value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                out = model(value,ts) # [batch,784]
                # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                loss = (epss-out).pow(2).mean(dim=-1).mean(dim=0)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
                bar.set_description('epoch {}, loss {:.4f}'.format(epoch,sum(losses)/len(losses)))
        model.eval()
        with torch.no_grad():
            with tqdm(valid_loader) as bar:
                mses = []
                losses = []
                for x,_ in bar:
                    x = pre_process(x.to(device))
                    epss = torch.randn_like(x).reshape(-1,784).to(device)
                    ts = torch.randint(0,T,(x.shape[0],),device=device,dtype=torch.long)
                    alpha_tbars = alpha_bars[ts]
                    value = (sqrt(alpha_tbars).reshape(-1,1,1,1)*x).reshape(-1,784)+sqrt(1-alpha_tbars).reshape(-1,1)*epss
                    out = model(value,ts)
                    mse = F.mse_loss(epss,out)
                    mses.append(mse.item())
                    # loss = ((epss-out).pow(2).mean(dim=-1) * (betas[ts])/(2*alphas[ts]*(1-alpha_tbars))).sum(dim=0)
                    loss = (epss-out).pow(2).mean(dim=-1).mean(dim=0)
                    losses.append(loss.item())
                    bar.set_description('epoch {}, MSE {:.4f}, [Valid] {:.4f}'.format(epoch,sum(mses)/len(mses),sum(losses)/len(losses)))
                    
        if epoch % eval_interval == 0:
            visualize(model,save_dir=os.path.join('./samples',f'diffuse_epoch_{epoch}.png'))
            sample(model,save_dir=os.path.join('./samples',f'sample_epoch_{epoch}.png'))
            torch.save(model,os.path.join('./samples',f'epoch_{epoch}.pt'))

if __name__ == '__main__':
    model = DDPM().to(device)
    print('Number parameters of the model:', sum(p.numel() for p in model.parameters()))
    print('Model strcuture:',model)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    os.makedirs('./samples',exist_ok=True)
    sample(model,save_dir=os.path.join('./samples',f'init.png'))
    visualize(model,save_dir=os.path.join('./samples',f'init_visualize.png'))
    train(200,model,optimizer,eval_interval=5)

appended /home/zhh24/DeepLearning
tensor([0.9997, 0.9994, 0.9990, 0.9986, 0.9981, 0.9976, 0.9971, 0.9965, 0.9959,
        0.9952, 0.9945, 0.9938, 0.9930, 0.9922, 0.9914, 0.9905, 0.9896, 0.9886,
        0.9876, 0.9866, 0.9855, 0.9844, 0.9832, 0.9821, 0.9808, 0.9796, 0.9783,
        0.9769, 0.9756, 0.9742, 0.9727, 0.9712, 0.9697, 0.9682, 0.9666, 0.9649,
        0.9633, 0.9616, 0.9598, 0.9581, 0.9563, 0.9544, 0.9526, 0.9507, 0.9487,
        0.9468, 0.9448, 0.9427, 0.9407, 0.9386, 0.9364, 0.9343, 0.9321, 0.9298,
        0.9276, 0.9253, 0.9230, 0.9206, 0.9182, 0.9158, 0.9134, 0.9109, 0.9084,
        0.9058, 0.9033, 0.9007, 0.8981, 0.8954, 0.8928, 0.8901, 0.8873, 0.8846,
        0.8818, 0.8790, 0.8762, 0.8733, 0.8704, 0.8675, 0.8646, 0.8616, 0.8587,
        0.8557, 0.8526, 0.8496, 0.8465, 0.8434, 0.8403, 0.8372, 0.8340, 0.8308,
        0.8276, 0.8244, 0.8211, 0.8179, 0.8146, 0.8113, 0.8080, 0.8047, 0.8013,
        0.7979, 0.7945, 0.7911, 0.7877, 0.7843, 0.7808, 0.7773, 0.7739, 0.7704,
      

epoch 0, loss 16.9689:   1%|█                                                                                                     | 1/94 [00:00<00:17,  5.20it/s]

Saved visualize to /home/zhh24/samples/init_visualize.png


epoch 0, MSE 0.2866, [Valid] 0.2866: 100%|███████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.62it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_0.png


...

epoch 190, MSE 0.0169, [Valid] 0.0169: 100%|█████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.60it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_190.png


epoch 195, MSE 0.0163, [Valid] 0.0163: 100%|█████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.56it/s]


Saved visualize to /home/zhh24/samples/diffuse_epoch_195.png


epoch 199, MSE 0.0173, [Valid] 0.0173: 100%|█████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  9.56it/s]
